In [1]:
import os
import math
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
os.chdir ('Z:\PPMI_Data\Excels\CollaborativeFiltering')
df = pd.read_csv('Feats45_unCategAge_APPRDX.csv')
df1 = df.drop(['PATNO','APPRDX'], axis = 1)
df2 = pd.read_csv('NonMotor_Empty.csv')
df3 = df2.drop('Patient_ID', axis = 1)
new_age = int(input("What is the user's age (in years): "))
df3.at[0, 'Age'] = new_age
col_age = df3.pop('Age')
df3.insert(0, "Age", col_age)

def update_values(df3):
    # Input from the user for number of columns
    num_columns = int(input("Enter the number of columns you want to update (minimum 3, maximum 45): "))
    
    while num_columns < 3 or num_columns > 45:
        print("Invalid number of columns. Please enter a number between 3 and 45.")
        num_columns = int(input("Enter the number of columns you want to update (minimum 3, maximum 45): "))
    
    columns = []
    values = []
     
     # Loop to get input from the user for each column
    for i in range(num_columns):
        column = input("Enter the name of the Feature %d: " % (i + 1))
        value = input("Enter the score of the Feature %d: " % (i + 1))
        
        columns.append(column)
        values.append(int(value))
    
    # Update the values in the dataframe
    for i in range(num_columns):
        df3.loc[df3[columns[i]] != values[i], columns[i]] = values[i]
    
    return df3

df3 = update_values(df3)
df3.insert (0, 'Patient_ID', df2['Patient_ID'])
merged_df = pd.concat ([df1, df3], axis = 0)
merged_df.reset_index(inplace = True)
merged_df1 = merged_df.drop(['index','Patient_ID'], axis = 1)
scaler = MinMaxScaler (feature_range = (1,5))
df_scaled = scaler.fit_transform(merged_df1)
df_scaled = pd.DataFrame (df_scaled, columns = merged_df1.columns)
df_scaled.insert (0, 'Patient_ID', merged_df['Patient_ID'], True)
df_melted = df_scaled.melt(id_vars='Patient_ID', var_name='Feature', value_name='Value')
feat_rating_count = df_melted.groupby('Patient_ID')['Value'].count()
feat_rating_count = pd.DataFrame(feat_rating_count)
feat_rating_count.columns = ['Value_count']
feat_rating_count['Patient_ID'] = feat_rating_count.index
feat_rating_count = feat_rating_count.reset_index(drop=True)
dframe = df_melted.merge(feat_rating_count, on = 'Patient_ID', how = 'inner')
RatingMat = dframe.pivot_table(index=['Patient_ID'],columns=['Feature'],values=['Value'],fill_value=0)
Original_RatingMat = RatingMat.copy()
RatingMat.columns = RatingMat.columns.droplevel()
RatingMat_centered = RatingMat.sub(RatingMat.mean(axis=1), axis=0)
user_similarity = cosine_similarity(RatingMat_centered)
user_sim_df = pd.DataFrame(user_similarity, index=RatingMat.index, columns=RatingMat.index)
user_Eucsimilarity = 1 - pairwise_distances(RatingMat, metric="euclidean")
user_Eucsim_df = pd.DataFrame(user_Eucsimilarity, index=RatingMat.index, columns=RatingMat.index)
current_user_rating = dframe[(dframe.Patient_ID == 999) & (dframe.Value != 0)]['Feature']
current_user_rating = pd.DataFrame(current_user_rating, columns=['Feature'])
curr_user_similarity = pd.DataFrame(user_sim_df.loc[999])
curr_user_similarity.rename(columns={999:'Similarity_Score'},inplace=True)
curr_user_similarity.sort_values(by='Similarity_Score', ascending=False, inplace=True)
curr_user_similarity1 = curr_user_similarity.iloc[1:, :]
curr_user_similarity1.reset_index(inplace=True)
curr_user_similarity1.rename(columns={'index': 'Index_Column'}, inplace=True)

def categorize(x):
    if x < 165:
        return 'Patient'
    else:
        return 'Healthy'

curr_user_similarity1['Patient_Type'] = curr_user_similarity1['Patient_ID'].apply(categorize)
similar_user = curr_user_similarity1.iloc[:5, :]
counts = similar_user['Patient_Type'].value_counts()
most_occurring_value = counts.index[0]
print("The user might fall under category of", most_occurring_value)

What is the user's age (in years): 66
Enter the number of columns you want to update (minimum 3, maximum 45): 3
Enter the name of the Feature 1: UPSIT
Enter the score of the Feature 1: 31
Enter the name of the Feature 2: Clock
Enter the score of the Feature 2: 4
Enter the name of the Feature 3: Pain
Enter the score of the Feature 3: 2
The user might fall under category of Healthy


C:\Users\system 4\AppData\Local\Temp\ipykernel_9568\3086389099.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_user_similarity1.rename(columns={'index': 'Index_Column'}, inplace=True)
C:\Users\system 4\AppData\Local\Temp\ipykernel_9568\3086389099.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_user_similarity1['Patient_Type'] = curr_user_similarity1['Patient_ID'].apply(categorize)


### Predicting Scoring of user: Top users of its category

In [26]:
select_simil_user = similar_user.loc[similar_user['Patient_Type'] == most_occurring_value]
df_simil = df1[df1['Patient_ID'].isin(select_simil_user['Patient_ID'])]
similar_user = similar_user.drop(['Patient_Type'], axis=1)
df_score = pd.merge(similar_user, df_simil, on = 'Patient_ID', how = 'inner')
X = df_simil
Y = df3.drop(['Patient_ID', 'Age'], axis = 1)
weights = df_score.iloc[:, 1]
values = X.iloc[:, 2:]
weighted_avg_cols = np.average(values, axis=0, weights=weights)
fill_values = {col: val for col, val in zip(Y.columns, weighted_avg_cols)}
Y = Y.fillna(fill_values)
col_Age = df3.pop('Age')
Y.insert(0, 'Age', col_Age)
Y